In [ ]:
faiss是否保留 metadata - 加载索引

In [1]:
%%time
%%capture

# 所有依赖库
!pip install llama-index
!pip install llama-index-llms-openai-like
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-faiss

CPU times: user 30.4 ms, sys: 8.74 ms, total: 39.1 ms
Wall time: 7.42 s


In [2]:
%%time

# 导入需要的包

from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

CPU times: user 1.37 s, sys: 225 ms, total: 1.59 s
Wall time: 1.21 s


In [3]:
%%time

# 初始化全局 embedding 模型
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 619 ms, sys: 35.2 ms, total: 654 ms
Wall time: 654 ms


In [4]:
%%time

import logging
import sys

# 设置系统日志，便于设置level排查

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 122 µs, sys: 15 µs, total: 137 µs
Wall time: 140 µs


In [5]:
%%time

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context)

INFO:faiss.loader:Loading faiss with AVX2 support.
Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
Successfully loaded faiss with AVX2 support.
INFO:root:Loading llama_index.vector_stores.faiss.base from ./storage/default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage/default__vector_store.json.
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
CPU times: user 218 ms, sys: 23.2 ms, total: 241 ms
Wall time: 239 ms


In [6]:
%%time

from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True
                )
Settings.llm =llm

CPU times: user 1.09 s, sys: 140 ms, total: 1.23 s
Wall time: 1.23 s


In [7]:
%%time

# set Logging to DEBUG for more detailed outputs
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=10,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("嫦娥六号探测器在哪里着陆")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
嫦娥六号探测器在月球背面南极-艾特肯盆地预选着陆和采样。
CPU times: user 127 ms, sys: 19.9 ms, total: 147 ms
Wall time: 1.1 s


In [8]:
streaming_response

StreamingResponse(response_gen=<generator object stream_chat_response_to_tokens.<locals>.gen at 0x7f24e444a880>, source_nodes=[NodeWithScore(node=TextNode(id_='11a65fd7-56a4-496c-9b4a-23221143699a', embedding=None, metadata={'file_path': '/root/notebook/my-jupyter-notebook/llm/faiss/news/n1.txt', 'file_name': 'n1.txt', 'file_type': 'text/plain', 'file_size': 7003, 'creation_date': '2024-05-07', 'last_modified_date': '2024-05-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='aa05b489-d81c-4a34-b022-7c35eddffdbd', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/root/notebook/my-jupyter-notebook/llm/faiss/news/n1.txt', 'file_name': 'n1.txt', 'file_type': 'text/plain', 'file_size': 7003, 'cre